# About

keras-bert

## Setup


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
pip install keras-bert


  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=47a212e17a9947dada5c3e1bfca7fd857052f507b03caedfe32964097b971e35
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=ac5aa4a8136b6d4498d4d3d2e539fb94789ee3323cb1f7217006e0149af48320
  Stored in directory: /root/.cache/pip/wheels/e5/fb/3a/37b2b9326c799aa010ae46a04ddb04f320d8c77c0b7e837f4e
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=ebaf75a8a870bc848652f1498db5493f9f2e324efe63f1a689499900e5893083
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=815242b8ebecf413774fb17333f14577e85389b097e9f78722292d57c5b147a5
  Stor

## prepare dataset


In [ ]:
vocab_size = 283 # includes special tokens
# Pretend maxlen is 20
maxlen = 100

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/train_multiclass.csv")
valid = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/valid_multiclass.csv")
test = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/test_multiclass.csv")

In [ ]:
def split_and_convert(row):

  return [int(x) for x in row.split(" ")]


In [ ]:
train["calls"] = train["calls"].apply(lambda x: split_and_convert(x))
valid["calls"] = valid["calls"].apply(lambda x: split_and_convert(x))
test["calls"] = test["calls"].apply(lambda x: split_and_convert(x))


In [ ]:
y_train = np.asarray(train["label"])
x_train = np.stack(np.asarray(train["calls"]),axis=0)

y_val = np.asarray(valid["label"])
x_val = np.stack(np.asarray(valid["calls"]),axis=0)


In [ ]:
y_test = np.asarray(test["label"])
x_test = np.stack(np.asarray(test["calls"]),axis=0)

## pre-train a BERT model on our data



In [ ]:
from random import randrange

sep_id = randrange(10,90)

training_input = []

for _, row in train[["calls"]].iterrows():
  sep_id = randrange(10,90)
  row_value = row[0]
  list_input = [row_value[:sep_id], row_value[sep_id:]]
  training_input.append(list_input)


In [ ]:
import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs


# Use training_input

In [ ]:

### SET MAX LEN
maxlen = 100

# Build token dictionary
token_dict = get_base_dict()   # A dict that contains some special tokens
for pairs in training_input:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word
len(token_dict)

253

In [ ]:
# # Ensure token dict has all the required tokens
## ONLY NEEDED IF TRIMMING
# for token in x_train.flatten():
#   if token not in token_dict:
#     token_dict[token] = len(token_dict)
# token_list = list(token_dict.keys())  # Used for selecting a random word

In [ ]:


# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=2,
    transformer_num=2,
    embed_dim=20,
    feed_forward_dim=25,
    seq_len=maxlen,
    pos_num=maxlen,
    dropout_rate=0.05,
)
compile_model(model)
model.summary()


Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 20), (2 5060        Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 20)      40          Input-Segment[0][0]              
_______________________________________________________________________________________

In [ ]:

def _generator():
    while True:
        yield gen_batch_inputs(
            training_input,
            token_dict,
            token_list,
            seq_len=maxlen,
            mask_rate=0.3,
            swap_sentence_rate=0.0, # don't apply sentence swapping
        )

model.fit_generator(
    generator=_generator(),
    steps_per_epoch=10,
    epochs=3,
    validation_data=_generator(),
    validation_steps=5,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)

Epoch 1/3


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


10/10 [==============================] - 125s 12s/step - loss: 1.9661 - MLM_loss: 1.6284 - NSP_loss: 0.3377 - val_loss: 1.9476 - val_MLM_loss: 1.6308 - val_NSP_loss: 0.3168
Epoch 2/3
10/10 [==============================] - 124s 12s/step - loss: 1.9688 - MLM_loss: 1.6304 - NSP_loss: 0.3384 - val_loss: 1.9469 - val_MLM_loss: 1.6307 - val_NSP_loss: 0.3161
Epoch 3/3
10/10 [==============================] - 123s 12s/step - loss: 1.9663 - MLM_loss: 1.6282 - NSP_loss: 0.3380 - val_loss: 1.9458 - val_MLM_loss: 1.6302 - val_NSP_loss: 0.3156


In [ ]:

# # Use the trained model
# inputs, output_layer = get_model(
#     token_num=len(token_dict),
#     head_num=5,
#     transformer_num=12,
#     embed_dim=25,
#     feed_forward_dim=100,
#     seq_len=maxlen,
#     pos_num=maxlen,
#     dropout_rate=0.05,
#     training=False,      # The input layers and output layer will be returned if `training` is `False`
#     trainable=False,     # Whether the model is trainable. The default value is the same with `training`
#     output_layer_num=4,  # The number of layers whose outputs will be concatenated as a single output.
#                          # Only available when `training` is `False`.
# )

## Fine tune the model

In [ ]:
# Make a copy of the model
classification_model = model

In [ ]:
classification_model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 20), (2 5060        Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 20)      40          Input-Segment[0][0]              
_______________________________________________________________________________________

In [ ]:
inputs = classification_model.inputs[:2]
dense = classification_model.get_layer('NSP-Dense').output
outputs = keras.layers.Dense(units=8, activation='softmax')(dense)

classification_model = keras.models.Model(inputs, outputs)
classification_model.compile(
    "adam",
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy', 'accuracy'],
)

In [ ]:
classification_model.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 20), (2 5060        Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 20)      40          Input-Segment[0][0]              
______________________________________________________________________________________

## Prepare input data

In [ ]:
def apply_tokenizer(x):

  try:
    return TOKEN_DICT[x]
  except:
    # Assign unknown
    print(f"assigning unknown to {x}")
    return 1

In [ ]:
# Apply tokenizer to data
# Super hacky!
# Need to figure out how to tokenize everything properly

TOKEN_DICT = token_dict

def split_convert_tokenize(row):

  seq =  [int(x) for x in row.split(" ")]

  return [apply_tokenizer(x) for x in seq]

train = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/train_multiclass.csv")
valid = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/valid_multiclass.csv")
test = pd.read_csv("/content/drive/My Drive/Research/CyberBERT/data/test_multiclass.csv")

train["calls"] = train["calls"].apply(lambda x: split_convert_tokenize(x))
valid["calls"] = valid["calls"].apply(lambda x: split_convert_tokenize(x))
test["calls"] = test["calls"].apply(lambda x: split_convert_tokenize(x))


y_train = np.asarray(train["label"])
x_train = np.stack(np.asarray(train["calls"]),axis=0)

y_val = np.asarray(valid["label"])
x_val = np.stack(np.asarray(valid["calls"]),axis=0)

y_test = np.asarray(test["label"])
x_test = np.stack(np.asarray(test["calls"]),axis=0)


assigning unknown to 260
assigning unknown to 237
assigning unknown to 237
assigning unknown to 239
assigning unknown to 239
assigning unknown to 239
assigning unknown to 257
assigning unknown to 257
assigning unknown to 257
assigning unknown to 257
assigning unknown to 257
assigning unknown to 239
assigning unknown to 239
assigning unknown to 239


In [ ]:
def format_data(x_train):
    return [x_train, np.zeros_like(x_train)]

In [ ]:
x_train_formatted = format_data(x_train)
x_val_formatted = format_data(x_val)
x_test_formatted = format_data(x_test)

In [ ]:
history = classification_model.fit(
    x_train_formatted, y_train, epochs=10, batch_size=maxlen, validation_data=(x_val_formatted,y_val)
)

Epoch 1/10
40/40 [==============================] - 11s 280ms/step - loss: 2.0744 - sparse_categorical_accuracy: 0.1807 - accuracy: 0.1807 - val_loss: 1.9672 - val_sparse_categorical_accuracy: 0.2588 - val_accuracy: 0.2588
Epoch 2/10
40/40 [==============================] - 10s 250ms/step - loss: 1.9124 - sparse_categorical_accuracy: 0.2892 - accuracy: 0.2892 - val_loss: 1.8691 - val_sparse_categorical_accuracy: 0.3069 - val_accuracy: 0.3069
Epoch 3/10
40/40 [==============================] - 10s 261ms/step - loss: 1.8050 - sparse_categorical_accuracy: 0.3305 - accuracy: 0.3305 - val_loss: 1.8263 - val_sparse_categorical_accuracy: 0.3358 - val_accuracy: 0.3358
Epoch 4/10
40/40 [==============================] - 10s 258ms/step - loss: 1.7053 - sparse_categorical_accuracy: 0.3825 - accuracy: 0.3825 - val_loss: 1.7677 - val_sparse_categorical_accuracy: 0.3681 - val_accuracy: 0.3681
Epoch 5/10
40/40 [==============================] - 10s 254ms/step - loss: 1.6259 - sparse_categorical_accur

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Dense, Dropout, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SpatialDropout1D
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt 

In [ ]:
y_test_pred = classification_model.predict(x_test_formatted)
y_classes = y_test_pred.argmax(axis=-1)
cm = confusion_matrix(y_test, y_classes)

plot_confusion_matrix(conf_mat=cm,
                      show_absolute=True,
                      show_normed=True,
                      colorbar=True)
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.savefig("accuracy.png")
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid()
plt.savefig("loss.png")
plt.show()